In [ ]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import json

from bouter import EmbeddedExperiment
from bouter.utilities import predictive_tail_fill
TURN_BIAS = 0.239

In [ ]:
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v28\2d")
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v29_29p1s\2sec")
fish_list = list(master_path.glob("*_f*"))

In [ ]:
for fish in fish_list:
    print(fish)
    try:
        if not (fish / "bouts_df.h5").exists():
            exp = EmbeddedExperiment(fish)
            beh_df = exp.behavior_log
            #theta_mat = beh_df.loc[:, [f"theta_0{i}" for i in range(9)]].values
            theta_mat = beh_df.iloc[:, 1:-1].values

            n_segments = np.shape(theta_mat)[1]
            beh_df.loc[:, [f"theta_0{i}" for i in range(n_segments)]] = predictive_tail_fill(
                theta_mat
            )

            try:
                beh_df["tail_sum"] = (beh_df["theta_07"] + beh_df["theta_08"]) - (
                    beh_df["theta_00"] + beh_df["theta_01"]
                )
            except:
                try:
                    beh_df["tail_sum"] = (beh_df["theta_07"] + beh_df["theta_06"]) - (
                        beh_df["theta_00"] + beh_df["theta_01"]
                    )
                except:
                    beh_df["tail_sum"] = (beh_df["theta_05"] + beh_df["theta_04"]) - (
                    beh_df["theta_00"] + beh_df["theta_01"]
                )

            bouts_df = exp.get_bout_properties()
            # Compute bout index in behavior trace:
            bouts_df["idx"] = [
                np.argmin((beh_df["t"] - bouts_df.loc[i, "t_start"]).abs())
                for i in bouts_df.index
            ]
            bouts_df["fid"] = fish.name

            #bouts_df["idx_imaging"] = np.round(bouts_df["t_start"] * fn).astype(np.int)

            bouts_df["direction"] = "fw"
            bouts_df.loc[(bouts_df["bias"] > TURN_BIAS), "direction"] = "rt"
            bouts_df.loc[(bouts_df["bias"] < -TURN_BIAS), "direction"] = "lf"

            fl.save(fish / "bouts_df.h5", bouts_df)
    except:
        print("Error. Stupid fish wasting my time.")

In [ ]:
theta_mat = beh_df.iloc[:, 1:-1]

In [ ]:
beh_df

In [ ]:
bouts_df